In [57]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [58]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)
x_train = mnist.train.images
y_train = mnist.train.labels
x_test = mnist.test.images
y_test = mnist.test.labels

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [59]:
#for NN layers
class layer:
    def __init__(self, inputs, in_size, out_size, activation_function=None):
#         self.W = tf.Variable(tf.zeros([in_size, out_size]))
        self.W = tf.Variable(tf.random_normal([in_size, out_size]))
        self.b = tf.Variable(tf.zeros([1,out_size]))
        self.Wx_plus_b = tf.matmul(inputs, self.W) + self.b
        self.activation_function = activation_function
    def output(self):
        if self.activation_function == None:
            result = self.Wx_plus_b
        else :
            result = self.activation_function(self.Wx_plus_b)
        return result

In [60]:
# for convolution
class convolution:
    def __init__(self, shape):
        self.W = tf.Variable(tf.truncated_normal(shape=shape, stddev=0.1))
        self.b = tf.Variable(tf.constant(0.1, shape=[shape[3]]))
    def conv2d(self, inputs, padding='SAME'):
        return tf.nn.conv2d(inputs, self.W, strides=[1,1,1,1], padding=padding)
    def max_pooling_nxn(self, inputs, n):
        return tf.nn.max_pool(inputs, ksize=[1,n,n,1], strides=[1,n,n,1], padding='SAME')
    def conv_and_pooling(self, inputs, activation_function=None, pooling_size=2):
        if activation_function==None:
            h_conv =self.conv2d(inputs)+self.b
        else:
            h_conv = activation_function(self.conv2d(inputs)+self.b)    
        return self.max_pooling_nxn(h_conv, pooling_size)

In [61]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

#for dropout
keep_prob = tf.placeholder(tf.float32)

#dealing eith inputs. 
# -1 for not considering input number of images
# 28,28 size per iamge
# 1 for channel. 1 for grayscale ; 3 for color image
x_image=tf.reshape(x,[-1,28,28,1])

In [62]:
#AlexNet => 5 convolution layers & 3 fully connected neural network

conv1 = convolution([3,3,1,12])
conv2 = convolution([3,3,12,24])
conv3 = convolution([3,3,24,48])
# conv4 = convolution([3,3,80,80])
# conv5 = convolution([3,3,80,80])

output_conv1 = conv1.conv_and_pooling(x_image, tf.nn.relu)
output_conv2 = conv2.conv_and_pooling(output_conv1, tf.nn.relu)
output_conv3 = conv3.conv_and_pooling(output_conv2, tf.nn.relu, pooling_size=2)
# output_conv4 = conv4.conv_and_pooling(output_conv3, tf.nn.relu, pooling_size=1)
# output_conv5 = conv5.conv_and_pooling(output_conv4, tf.nn.relu, pooling_size=1)

# h_pool_flat = tf.reshape(output_conv2, [-1,7*7*40])
h_pool_flat = tf.reshape(output_conv3, [-1,4*4*48])
# layer1 = layer(h_pool_flat, 7*7*64, 500, tf.nn.relu)
# layer1 = layer(h_pool_flat, 7*7*64, 10, tf.nn.softmax)
# layer1 = layer(h_pool_flat, 7*7*40, 10, tf.nn.softmax)
layer1 = layer(h_pool_flat, 4*4*48, 100, tf.nn.sigmoid)
# layer2 = layer(layer1.output(), 400, 100, tf.nn.sigmoid)
h_drop = tf.nn.dropout(layer1.output(), keep_prob)
layer3 = layer(h_drop, 100, 10, tf.nn.softmax)
# layer2 = layer(layer1.output(), 500, 100, tf.nn.relu)
# layer3 = layer(layer2.output(), 100, 10, tf.nn.softmax)

# loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=layer1.output()))
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=layer3.output()))
# train_step = tf.train.GradientDescentOptimizer(0.3).minimize(loss)
# train_step =  tf.train.MomentumOptimizer(0.005 , 0.9).minimize(loss)
train_step = tf.train.AdamOptimizer(0.003).minimize(loss)


correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(layer3.output(), 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [63]:
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

batch_size = 100
batches = x_train.shape[0]//batch_size


In [64]:
for epoch in range(51):
    print "start "+str(epoch)
    for batch in range(batches):
        sess.run(train_step, feed_dict={x: x_train[batch_size*batch:batch_size*(batch+1)], y: y_train[batch_size*batch:batch_size*(batch+1)], keep_prob: 0.5})
    if epoch%2==0:
        print "epoch: "+str(epoch)+" loss: "+str(sess.run(loss, feed_dict={x: x_train, y: y_train, keep_prob: 0.5}))+" accuracy: "+str(sess.run(accuracy, feed_dict={x: x_train, y: y_train, keep_prob: 0.5}))
print "Test accuracy: "+str(sess.run(accuracy, feed_dict={x: x_test, y: y_test, keep_prob: 0.3}))

start 0
epoch: 0 loss: 2.10437 accuracy: 0.353509
start 1
start 2
epoch: 2 loss: 1.65818 accuracy: 0.805473
start 3
start 4
epoch: 4 loss: 1.61704 accuracy: 0.843927
start 5
start 6
epoch: 6 loss: 1.54559 accuracy: 0.9184
start 7
start 8
epoch: 8 loss: 1.50826 accuracy: 0.954673
start 9
start 10
epoch: 10 loss: 1.50698 accuracy: 0.955509
start 11
start 12
epoch: 12 loss: 1.50469 accuracy: 0.957182
start 13
start 14
epoch: 14 loss: 1.50037 accuracy: 0.962291
start 15
start 16
epoch: 16 loss: 1.50213 accuracy: 0.960945
start 17
start 18
epoch: 18 loss: 1.49176 accuracy: 0.968545
start 19
start 20
epoch: 20 loss: 1.49216 accuracy: 0.969691
start 21
start 22
epoch: 22 loss: 1.49152 accuracy: 0.970509
start 23
start 24
epoch: 24 loss: 1.49454 accuracy: 0.965509
start 25
start 26
epoch: 26 loss: 1.48981 accuracy: 0.971764
start 27
start 28
epoch: 28 loss: 1.496 accuracy: 0.965327
start 29
start 30
epoch: 30 loss: 1.49187 accuracy: 0.968327
start 31
start 32
epoch: 32 loss: 1.49224 accuracy: 